In [34]:
"""
Downloading data from https://materials.hybrid3.duke.edu/

Database have atomic structure as a *.cif file for 528 different materials. Also, it contains some other physical
properties such as Photoluminescence or band gap. This script demonstrate how to download some dataset from website
"""


import requests, pprint

In [36]:
# The list of avalibale materials can be downloaded as next;
responseURL = requests.get('https://materials.hybrid3.duke.edu/materials/systems/')
dictMaterial = responseURL.json()

print("Total number of material:", dictMaterial['count'])

Total number of material: 528


In [37]:
idxMaterial = 442  #put any number between [1, 528]


numMaterial = dictMaterial['results'][idxMaterial]['pk']
url = 'https://materials.hybrid3.duke.edu/materials/{}'.format(numMaterial)
print("the url with API for material under number {} is avaliabele at:\n".format(idxMaterial), url)

print("\nExample of info about material number:", idxMaterial)
for k, v in (dictMaterial['results'][idxMaterial]).items():
    print(k, ':\t', v)

the url with API for material under number 442 is avaliabele at:
 https://materials.hybrid3.duke.edu/materials/92

Example of info about material number: 442
pk :	 92
compound_name :	 Bis(1-butylammonium) bis(methylammonium) lead iodide
formula :	 C10H36N4Pb3I10
group :	 bis(butane-1-aminium) di(methanaminium) decaiodo triplumbate(II), (BA)2(MA)2Pb3I10
organic :	 C4H12N, CH6N
inorganic :	 Pb3I10, Lead iodide
iupac :	 bis(butane-1-aminium) di(methanaminium) lead (II) iodide
last_update :	 2022-10-18
derived_to_from :	 []
description :	 


In [38]:
# To get avaliable dataset for material we need request it as follows:
url='https://materials.hybrid3.duke.edu/materials/datasets/?system={}'.format(numMaterial)
responseURL = requests.get(url)
dataset = responseURL.json()

print("\nThere are {} results for this material:".format(len(dataset['results'])))
for i in dataset['results']:
    if i['secondary_property'] is None:
        print(i['primary_property']['name'])
    else:
        print(i['primary_property']['name'], ' vs ', i['secondary_property']['name'])


There are 6 results for this material:
Exciton energy
exciton binding energy
band gap (fundamental)
photoluminescence  vs  wavelength
photoluminescence  vs  wavelength
atomic structure


In [39]:
# if property is not atomic structure, then the data are contained in dataset and we can print them
print("\n Data for specific property:")
for i in dataset['results']:
    url = 'https://materials.hybrid3.duke.edu/materials/datasets/{}/files/'.format(i['pk'])
    if i['secondary_property'] is None:
        unit1 = i['primary_unit']['label']
        print("\n (Primary) {}({}). Data also avaliable at url: {}".format(i['primary_property']['name'], unit1, url))
    else:
        unit1, unit2 = i['primary_unit']['label'], i['secondary_unit']['label']
        print('\n (Primary) {}({})  vs (Secondary) {}({}). Data also avaliable at url: {}'.format(
            i['primary_property']['name'], unit1, i['secondary_property']['name'], unit2, url))

    if i['primary_property']['name'] != 'atomic structure':
        datalist = []
        print("Data for this property")
        for x in i['subsets'][0]['datapoints']:
            vv = [v['formatted'] for v in x['values']]
            if x['values'][0]['qualifier'] == 'secondary':
                vv = vv[::-1]
            datalist.append(vv)
        pprint.pprint(datalist)

    else:
        print("Only *.cif file for atomic structure")





 Data for specific property:

 (Primary) Exciton energy(eV). Data also avaliable at url: https://materials.hybrid3.duke.edu/materials/datasets/2339/files/
Data for this property
[['2.043 (±0.015)']]

 (Primary) exciton binding energy(eV). Data also avaliable at url: https://materials.hybrid3.duke.edu/materials/datasets/2338/files/
Data for this property
[['0.077 (±0.025)']]

 (Primary) band gap (fundamental)(eV). Data also avaliable at url: https://materials.hybrid3.duke.edu/materials/datasets/2337/files/
Data for this property
[['2.12 (±0.02)']]

 (Primary) photoluminescence(a.u)  vs (Secondary) wavelength(nm). Data also avaliable at url: https://materials.hybrid3.duke.edu/materials/datasets/2126/files/
Data for this property
[['0.006602306115898404', '808.2294264339153'],
 ['0.004750499987654644', '800.1246882793017'],
 ['0.00490852077726478', '790.1496259351621'],
 ['0.005086294165576266', '778.927680798005'],
 ['0.005254191254536966', '768.3291770573567'],
 ['0.005422088343497888'